<a href="https://colab.research.google.com/github/woody35545/Biomarker/blob/main/CNN_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from os.path import join
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import StratifiedKFold

def sequence_to_onehot(_datax):
  res_list = []
  for i in range(len(_datax)):
    tmp =list()
    mapping = {"A":[1., 0., 0., 0.], "C": [0., 1., 0., 0.], "G": [0., 0., 1., 0.], "T":[0., 0., 0., 1.]}
    for j in _datax[i]:
      tmp.append(mapping[j]  if j in mapping.keys() else [0., 0., 0., 0.]) 
    res_list.append(np.array(tmp))
  res_np = np.array(res_list)
  return res_np

  # data shuffle
df_shuffled = pd.read_csv('data.csv').sample(frac=1).reset_index(drop=True)

# x, y 분류. numpy로 변환 후 1차원으로 reshape, sequence onehot encoding
datax = sequence_to_onehot(df_shuffled.iloc[:,[0,]].to_numpy().reshape(-1))
datay = df_shuffled.iloc[:,[2,]].to_numpy().reshape(-1)


'\n# Train, Test Split \ntrnx, tstx, trny, tsty = train_test_split(datax, datay, test_size = 0.3, random_state =111)\n\nprint(f"Train = {trnx.shape[0]}개, Test = {tstx.shape[0]}개")\nprint(f"trnx.shape = {trnx.shape}")\nprint(f"tstx.shape = {tstx.shape}")\nprint(f"trny.shape = {trny.shape}")\nprint(f"tsty.shape = {tsty.shape}")\n'

In [5]:
skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 7)


accuracy = []

for train, test in skf.split(datax, datay):
  input_shape = (5600,4,1)
  cnn_model = models.Sequential()

  cnn_model.add(layers.Conv2D(16,(2,2), padding='same', input_shape=input_shape))
  cnn_model.add(layers.BatchNormalization())
  cnn_model.add(layers.Activation("relu"))
  cnn_model.add(layers.MaxPooling2D((2,1)))

  cnn_model.add(layers.Conv2D(16,(2,2), padding='same'))
  cnn_model.add(layers.BatchNormalization())
  cnn_model.add(layers.Activation("relu"))
  cnn_model.add(layers.Dropout(0.2))
  cnn_model.add(layers.MaxPooling2D((2,1)))

  cnn_model.add(layers.Flatten())

  cnn_model.add(layers.Dense(units = 100, activation = "relu"))
  cnn_model.add(layers.Dense(units = 1, activation = "sigmoid"))

  opt = optimizers.SGD(learning_rate=0.01)
  cnn_model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
  cnn_model.fit(datax[train].reshape(-1,5600,4,1), datay[train], validation_data = [datax[test].reshape(-1,5600,4,1),datay[test]], batch_size=20, epochs = 100)

  k_accuracy = "%.4f" %(cnn_model.evaluate(datax[train],datay[train])[1])
  accuracy.append(k_accuracy)

print(f"\n %.f fold accuracy: " %(3), accuracy)

Epoch 1/100
9/9 [==============================] - 1s 93ms/step - loss: 26.4217 - accuracy: 0.5776 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 2/100
9/9 [==============================] - 0s 32ms/step - loss: 0.6933 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 3/100
9/9 [==============================] - 0s 30ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 4/100
9/9 [==============================] - 0s 23ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 5/100
9/9 [==============================] - 0s 23ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 6/100
9/9 [==============================] - 0s 23ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 7/100
9/9 [==============================] - 0s 23ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 8/100
9/9 [==

In [6]:
print(f"\n %.f fold accuracy: " %(3), accuracy)


 3 fold accuracy:  ['0.4969', '1.0000', '1.0000']
